# Compare runtime

In [1]:
#include("../netx.jl")
push!(LOAD_PATH, "../"); using netx
#import .netx

using BenchmarkTools

┌ Info: Precompiling netx [top-level]
└ @ Base loading.jl:1278


## Path to data

In [2]:
path_to = "../ER/"
folders = String["TYPE1", "TYPE2", "TYPE4", "TYPE8"]

sizes = Int[512, 1024, 2048, 4096, 8192, 16384]
k_mean = Float64[1.0, 2.0, 4.0, 8.0]
ntype = Int[1, 2, 4, 8]

4-element Array{Int64,1}:
 1
 2
 4
 8

In [58]:
function process_network(N::Int, k::Float64, ne_type::Int, samp::Int)
    fname = "$(path_to)TYPE$(ntype[ne_type])/ER_N$(N)_k$(k)_samp$(samp).txt"
    edges, eprops = netx.process_edgefile(fname, true)
    fmt_eprops = netx.process_eprops(eprops, ["regulation"])
    edgetype_prop = parse.(Int, fmt_eprops["regulation"])
    
    # -- Create network and set the edgetypes as an edge property --
    g = netx.graph_from_edgelist(edges, true)
    netx.set_edges_properties("edgetype", edgetype_prop, g)
    return g
end

function process_samples(N::Int, ne_type::Int)
    nsamp = 30
    runtime = [ Float64[0.0 for j in 1:nsamp] for i in 1:4 ]
    mem = [ Float64[0.0 for j in 1:nsamp] for i in 1:4 ]
    for (j, k_mean) in enumerate(Float64[1.0, 2.0, 4.0, 8.0])
        print("mean degree = $(k_mean)\n")
        for i in 1:nsamp
            print("$i ")
            g = process_network(N, k_mean, ne_type, i)
            b = @benchmarkable netx.fast_fibration($g)
            tune!(b)
            res = minimum(run(b))
            runtime[j][i] = res.time*1e-9
            mem[j][i] = res.memory*1e-6
        end
        print("\n")
    end
    return runtime, mem
end

process_samples (generic function with 1 method)

In [60]:
N512_T1_time, N512_T1_memory = process_samples(512, 1)
N1024_T1_time, N1024_T1_memory = process_samples(1024, 1)
N2048_T1_time, N2048_T1_memory = process_samples(2048, 1)
N4096_T1_time, N4096_T1_memory = process_samples(4096, 1)
N8192_T1_time, N8192_T1_memory = process_samples(8192, 1)
N16384_T1_time, N16384_T1_memory = process_samples(16384, 1)
N32768_T1_time, N32768_T1_memory = process_samples(32768, 1)
return

mean degree = 1.0
1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 
mean degree = 2.0
1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 
mean degree = 4.0
1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 
mean degree = 8.0
1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 


([[0.007714083, 0.008439586, 0.007638864, 0.008524784, 0.007376594, 0.007666862000000001, 0.008277083000000001, 0.008076110000000001, 0.008128842, 0.007688877  …  0.007558694, 0.008095467, 0.007763511000000001, 0.008160198, 0.007501067, 0.007821875, 0.008200693, 0.007904227, 0.007447726000000001, 0.008017392], [0.013562691, 0.014427348000000001, 0.014016383, 0.013699132000000001, 0.013970569, 0.013636294, 0.011959956, 0.013035485000000001, 0.013347310000000001, 0.013259777  …  0.013178426, 0.013429670000000001, 0.010246806, 0.014006123, 0.013760064, 0.013500266, 0.014347449000000002, 0.014223305, 0.013818078000000001, 0.013843194000000001], [0.015227076, 0.018624777000000002, 0.018838658, 0.017385897, 0.017851079000000002, 0.018284179, 0.018756563, 0.017659661, 0.018674237, 0.016206774  …  0.015608504, 0.015910404, 0.017538492000000003, 0.016863796, 0.016920098, 0.019134234, 0.018358359, 0.01993967, 0.017176073, 0.017549600000000002], [0.014625581, 0.021912412000000003, 0.0145138610000

In [65]:
N512_T1_memory[4]

30-element Array{Float64,1}:
 27.3936
 33.37
 26.831719999999997
 26.902583999999997
 35.385568
 26.712048
 26.813959999999998
 26.004016
 27.10764
 26.72376
 26.711152
 27.37396
 26.957687999999997
  ⋮
 26.870127999999998
 34.674448
 26.500152
 26.657144
 26.503256
 26.793736
 26.933328
 26.832824
 35.974584
 26.958688
 26.55012
 26.870535999999998

In [35]:
b = @benchmarkable netx.fast_fibration($g);
tune!(b);
out = run(b)

BenchmarkTools.Trial: 
  memory estimate:  316.64 MiB
  allocs estimate:  5334104
  --------------
  minimum time:     317.044 ms (20.48% GC)
  median time:      422.606 ms (19.50% GC)
  mean time:        412.871 ms (20.48% GC)
  maximum time:     457.473 ms (25.60% GC)
  --------------
  samples:          13
  evals/sample:     1

In [40]:
?out

search: out Out OutOfMemoryError stdout count count! countlines count_ones



No documentation found.

`out` is of type `BenchmarkTools.Trial`.

# Summary

```
mutable struct BenchmarkTools.Trial <: Any
```

# Fields

```
params  :: BenchmarkTools.Parameters
times   :: Array{Float64,1}
gctimes :: Array{Float64,1}
memory  :: Int64
allocs  :: Int64
```


In [41]:
out.gctimes

13-element Array{Float64,1}:
 6.4929507e7
 6.5049579e7
 7.7447115e7
 7.3743377e7
 7.8182339e7
 8.2424351e7
 7.983025e7
 9.0463789e7
 9.0254223e7
 9.9398462e7
 9.4692771e7
 8.5598788e7
 1.17096382e8

In [56]:
d = minimum(out)

BenchmarkTools.TrialEstimate: 
  time:             317.044 ms
  gctime:           64.930 ms (20.48%)
  memory:           316.64 MiB
  allocs:           5334104

In [54]:
t = (d.memory)

332016424

In [55]:
t*1e-6

332.016424